In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))
import itertools

import numpy as np
import bottleneck as bn
import pandas as pd
from sklearn import decomposition
import networkx as nx
# import networkx.algorithms.approximation as nxa
import igraph as ig
# # http://stackoverflow.com/questions/35279733/what-could-cause-networkx-pygraphviz-to-work-fine-alone-but-not-together
# from networkx.drawing.nx_agraph import graphviz_layout
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid", {"axes.facecolor": ".95"})
import missingno as msno
import palettable
%matplotlib inline
from io import StringIO

np.set_printoptions(linewidth=145)

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)
warnings.simplefilter("ignore", category=mpl.cbook.mplDeprecation)
warnings.simplefilter("ignore", category=UserWarning)


from collections import defaultdict, Counter
from IPython.display import display as disp
from IPython.display import display_markdown
mdisp = lambda s: display_markdown(s, raw=True)

from speclib.loaders import *
from speclib.plotting import *
from speclib.graph import *
from speclib.misc import *
from speclib.userActivityFunctions import *

for k, v in {'font.size': 13.0,
             'legend.fontsize': 13.0,
             'axes.labelsize': 12.0,
             'axes.titlesize': 15.0,
             'figure.figsize': [16.0, 7.0],
             'figure.dpi': 200,
             'figure.titlesize': 'large',
             'xtick.labelsize': 13.0,
             'ytick.labelsize': 13.0}.items():
    mpl.rcParams[k] = v

%load_ext watermark
%watermark -a "Allan Leander Rostock Hansen" -u -d -v -p numpy,bottleneck,pandas,matplotlib,sklearn,missingno
%watermark  -p networkx,igraph,seaborn,palettable

ModuleNotFoundError: No module named 'watermark'

In [2]:
from random import choice

In [3]:
N = 20

In [11]:
fstr = 'u{:0%dd}' % len(str(11))
[fstr.format(i) for i in range(11)]

['u00', 'u01', 'u02', 'u03', 'u04', 'u05', 'u06', 'u07', 'u08', 'u09', 'u10']

In [4]:
us = 'u1 u2 u3 u4 u5'.split()
data = list()
i = 0
while len(data) < N:
    u = choice(us)
    uss = list(set(us) - {u,})
    ch = choice(uss)
    tp = choice(['call', 'sms'])
    data.append([u, tp, ch])
data.sort(key=lambda x: x[0]) 
data

[['u1', 'call', 'u5'],
 ['u1', 'call', 'u3'],
 ['u1', 'call', 'u2'],
 ['u1', 'sms', 'u3'],
 ['u1', 'call', 'u3'],
 ['u2', 'call', 'u5'],
 ['u3', 'call', 'u2'],
 ['u3', 'call', 'u5'],
 ['u3', 'sms', 'u1'],
 ['u3', 'call', 'u4'],
 ['u3', 'call', 'u4'],
 ['u3', 'sms', 'u5'],
 ['u4', 'call', 'u2'],
 ['u4', 'call', 'u2'],
 ['u4', 'call', 'u3'],
 ['u4', 'call', 'u3'],
 ['u4', 'call', 'u2'],
 ['u4', 'sms', 'u3'],
 ['u5', 'call', 'u4'],
 ['u5', 'sms', 'u4']]

In [5]:
df = pd.DataFrame(data, columns='user comtype contactedUser'.split())
df = df.set_index(['user', 'comtype'], drop=False)
df

user comtype contactedUser
user comtype                           
u1   call      u1    call            u5
     call      u1    call            u3
     call      u1    call            u2
     sms       u1     sms            u3
     call      u1    call            u3
u2   call      u2    call            u5
u3   call      u3    call            u2
     call      u3    call            u5
     sms       u3     sms            u1
     call      u3    call            u4
     call      u3    call            u4
     sms       u3     sms            u5
u4   call      u4    call            u2
     call      u4    call            u2
     call      u4    call            u3
     call      u4    call            u3
     call      u4    call            u2
     sms       u4     sms            u3
u5   call      u5    call            u4
     sms       u5     sms            u4